# 1️⃣ Install and Verify libtorrent

In [ ]:
!python -m pip install --upgrade pip setuptools wheel -q

import sys
import subprocess

def try_install():
    try:
        import libtorrent
        print(" libtorrent already available!")
        return True
    except:
        pass

    print(" Installing libtorrent...")

    try:
        subprocess.run(['apt-get', 'update', '-qq'], check=True, capture_output=True)
        subprocess.run(['apt-get', 'install', '-y', '-qq',
                        'build-essential', 'libboost-all-dev',
                        'libssl-dev', 'python3-dev'],
                       check=True, capture_output=True)
        result = subprocess.run([sys.executable, '-m', 'pip', 'install',
                                'libtorrent', '--no-binary', ':all:'],
                               capture_output=True, text=True, timeout=300)
        import libtorrent
        print(" Successfully built from source!")
        return True
    except Exception as e:
        print(f" Source build failed: {str(e)[:100]}")

    try:
        subprocess.run([sys.executable, '-m', 'pip', 'install', 'libtorrent', '-q'],
                       check=True, capture_output=True)
        import libtorrent
        print(" Installed prebuilt package!")
        return True
    except:
        pass

    print(" libtorrent has issues with Python 3.12")
    print(" Solution: Runtime > Change runtime type > Python 3.10")
    return False

if try_install():
    print("\n All dependencies ready!")
else:
    sys.exit(1)


# 2️⃣ Initialize Torrent Session

In [ ]:
import libtorrent as lt
import time

ses = lt.session()
ses.listen_on(6881, 6891)

settings = {
    'user_agent': 'libtorrent/2.0',
    'listen_interfaces': '0.0.0.0:6881',
    'enable_outgoing_utp': True,
    'enable_incoming_utp': True,
    'enable_outgoing_tcp': True,
    'enable_incoming_tcp': True,
}
ses.apply_settings(settings)
ses.add_dht_router("router.utorrent.com", 6881)
ses.add_dht_router("router.bittorrent.com", 6881)
ses.add_dht_router("dht.transmissionbt.com", 6881)
ses.start_dht()
downloads = []

print(" libtorrent session initialized!")

# 3️⃣ Setup Local and Google Drive Paths

In [ ]:
import os
from google.colab import drive

# Stockage temporaire local Colab
local_path = "/content/torrents"
os.makedirs(local_path, exist_ok=True)

# Monter Drive
drive.mount('/content/drive')
drive_path = "/content/drive/My Drive/Torrents"
os.makedirs(drive_path, exist_ok=True)

print(f" Local path: {local_path}\n Drive path: {drive_path}")

# 4️⃣ Add Magnet and List Files

In [ ]:
magnet_link = input("🧲 Paste magnet link: ").strip()

params = {
    'save_path': local_path,  # Téléchargement local
    'storage_mode': lt.storage_mode_t.storage_mode_sparse,
}

handle = lt.add_magnet_uri(ses, magnet_link, params)
downloads.append(handle)
print("⏳ Fetching metadata...")

while not handle.has_metadata():
    time.sleep(1)

info = handle.get_torrent_info()
files = info.files()

print("\n📂 FILES IN TORRENT:\n")
for i in range(files.num_files()):
    size_gb = files.file_size(i) / (1024**3)
    print(f"[{i}] {files.file_path(i)} — {size_gb:.2f} GB")